In [1]:
# debug with epoch = 1
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import pickle

Using TensorFlow backend.


In [2]:
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [3]:
path = 'vae_models_structure/'
if not os.path.exists(path):
    os.mkdir(path)

# network parameters
input_shape = (784, )
latent_dim = 5

# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(512, activation='elu')(inputs)
x = Dense(384, activation='elu')(x)
x = Dense(256, activation='elu')(x)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()
plot_model(encoder, to_file=path+'model1_encoder.png', show_shapes=True)


# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(256, activation='elu')(latent_inputs)
x = Dense(384, activation='elu')(x)
x = Dense(512, activation='elu')(x)
outputs = Dense(784, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
# decoder.summary()
plot_model(decoder, to_file=path+'model1_decoder.png', show_shapes=True)


# instantiate VAE model
# outputs = decoder(encoder(inputs)[2])
# vae = Model(inputs, outputs, name='vae_mlp')
print ('done')

done


In [7]:
path = 'vae_models_structure/'
if not os.path.exists(path):
    os.mkdir(path)

    # network parameters
input_shape = (784, )
intermediate_dim = 196
latent_dim = 49
    
# VAE model = encoder + decoder
# build encoder model
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='sigmoid')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
# encoder.summary()
plot_model(encoder, to_file=path+'model3_encoder.png', show_shapes=True)


# build decoder model
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='sigmoid')(latent_inputs)
outputs = Dense(784, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
# decoder.summary()
plot_model(decoder, to_file=path+'model3_decoder.png', show_shapes=True)

print ('done')

done
